In [2]:
import torch
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader
from transformers import AutoConfig, AutoModelForSequenceClassification, BertTokenizer

import data_prep

In [3]:
torch.cuda.is_available()

True

In [3]:
pip uninstall torch

^C
Note: you may need to restart the kernel to use updated packages.


In [4]:
model_ckpt = "models/URLTran-BERT-CLS-0"
config = AutoConfig.from_pretrained(model_ckpt)
config.num_labels = 2
config.problem_type = "single_label_classification"

In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config)
device = torch.device("cuda")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\vishn\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vishn\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
torch.cuda.is_available()

True

In [7]:
def predict(url, tokenizer, model):
    inputs = data_prep.preprocess(url, tokenizer)
    return torch.argmax(torch.softmax(model(**inputs).logits, dim=1)).tolist()

In [8]:
def train_model(train_dataset, model):
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

    # model training
    model.to(device)
    model.train()

    # initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

    epochs = 1
    for epoch in range(epochs):
        for batch in train_loader:
            optimizer.zero_grad()
            # prep data for predict step
            inputs = batch["input_ids"]
            labels = batch["label"]
            X = inputs.to(device)
            y = labels.to(device)

            outputs = model(X, labels=y)

            loss = outputs.loss
            loss.backward()
            optimizer.step()

        print(f"Epoch: {epoch} Loss: {loss.item()}")
        model.save_pretrained(f"models/URLTran-BERT-CLS-{epoch}")

In [9]:
def eval_model(eval_dataset, tokenizer, model):
    eval_loader = DataLoader(eval_dataset, batch_size=2000, shuffle=True)

    y_true = []
    y_pred = []

    model.eval()
    with torch.no_grad():
        for batch in eval_loader:
            inputs = batch["input_ids"]
            labels = batch["label"]
            X_eval = inputs.to(device)
            y_eval = labels.to(device)

            outputs = model(X_eval, labels=y_eval)
            predictions = [
                torch.argmax(pred).tolist()
                for pred in torch.softmax(outputs.logits, dim=1)
            ]

            y_eval_true = y_eval.tolist()

            y_true.extend(y_eval_true)
            y_pred.extend(predictions)

        total_acc = accuracy_score(y_true, y_pred)
        total_f1 = f1_score(y_true, y_pred)
        print(f"Acc: {total_acc} F1: {total_f1}")

In [10]:
####Prediction
url = "http://amaz0n.pass-carod.ga"
inputs = tokenizer(url, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).tolist()
print(prediction)

[1]


In [11]:
data_path = "test_data.csv"
dataset = data_prep.URLTranDataset(data_path, tokenizer)

In [12]:
model.to(device)
eval_model(dataset, tokenizer, model)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


OutOfMemoryError: CUDA out of memory. Tried to allocate 750.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 6.28 GiB is allocated by PyTorch, and 43.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)